In [1]:
%pip install pandas matplotlib numpy


[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
csv_file = "./dataset/train.csv"
test_csv_file = "./dataset/test.csv"
train_raw_df = pd.read_csv(csv_file)
train_df = pd.read_csv(csv_file)
noise_df = pd.read_csv(csv_file)
train_null_df = pd.read_csv(test_csv_file)
dropList = ["Name","FoodCourt","RoomService",'ShoppingMall','Spa', 'VRDeck',"Age"]
train_df = train_df.drop(dropList, axis=1)
train_null_df = train_null_df.drop(dropList, axis=1)
train_raw_df = train_raw_df.drop(dropList, axis=1)
noise_df = noise_df.drop(dropList, axis=1)
# train_df = train_df.drop("Transported", axis=1)
print(train_df.shape)
print( train_df.columns)
print(train_df.info())
# print(train_df.isnull())
print(train_df.isnull().sum().sort_values(ascending=False))
# print(df.head())

(8693, 7)
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP',
       'Transported'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  8693 non-null   object
 1   HomePlanet   8492 non-null   object
 2   CryoSleep    8476 non-null   object
 3   Cabin        8494 non-null   object
 4   Destination  8511 non-null   object
 5   VIP          8490 non-null   object
 6   Transported  8693 non-null   bool  
dtypes: bool(1), object(6)
memory usage: 416.1+ KB
None
CryoSleep      217
VIP            203
HomePlanet     201
Cabin          199
Destination    182
PassengerId      0
Transported      0
dtype: int64


In [4]:
for key in train_df.columns:
    train_df = train_df[train_df[key].notna()]
    noise_df = noise_df[noise_df[key].notna()]
print(train_df.isnull().sum().sort_values(ascending=False))
print(train_df.shape)
print(train_df.columns)
print(train_df.info())
# print(train_df.isnull())
print(train_df.isnull().sum().sort_values(ascending=False))

PassengerId    0
HomePlanet     0
CryoSleep      0
Cabin          0
Destination    0
VIP            0
Transported    0
dtype: int64
(7736, 7)
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP',
       'Transported'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 7736 entries, 0 to 8692
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  7736 non-null   object
 1   HomePlanet   7736 non-null   object
 2   CryoSleep    7736 non-null   object
 3   Cabin        7736 non-null   object
 4   Destination  7736 non-null   object
 5   VIP          7736 non-null   object
 6   Transported  7736 non-null   bool  
dtypes: bool(1), object(6)
memory usage: 430.6+ KB
None
PassengerId    0
HomePlanet     0
CryoSleep      0
Cabin          0
Destination    0
VIP            0
Transported    0
dtype: int64


In [5]:
def onehot(df : pd.DataFrame,keyName : str) -> pd.DataFrame:
    for key in df[keyName].value_counts().index:
        newKey = f"{keyName}_{key}"
        df[newKey] = df[keyName] == key
        df[newKey] = df[newKey].astype(int)
    return df.drop(keyName, axis=1)
def mapValue(df : pd.DataFrame,keyName : str) -> pd.DataFrame:
    mapDict = {}
    for idx, key in enumerate(df[keyName].value_counts().index):
        mapDict[key] = idx
    print(mapDict)
    df[keyName] = df[keyName].map(mapDict)
    return df

In [6]:

def processData(df, isTrain = True):
    if isTrain:
        label = "Transported"
        df[label] = df[label].astype(int)
        df = df.drop(["PassengerId"], axis=1)
    # df["UnknowAge"] = df["Age"].isnull().astype(int)
    df["UnkownCabin"] = df["Cabin"].isnull().astype(int)
    df["UnkownHomePlanet"] = df["HomePlanet"].isnull().astype(int)
    df["UnkownDestination"] = df["Destination"].isnull().astype(int)
    df["UnknowVIP"] = df["VIP"].isnull().astype(int)
    df["UnkownCryoSleep"] = df["CryoSleep"].isnull().astype(int)
    fillnaList = ["VIP","CryoSleep","Cabin"]
    df[fillnaList] = df[fillnaList].fillna(0)
    df["HomePlanet"] = df["HomePlanet"].fillna("Earth")
    df["Destination"] = df["Destination"].fillna("TRAPPIST-1e")
    
    df['VIP'] = df['VIP'].astype(int)
    df['CryoSleep'] = df['CryoSleep'].astype(int)
    df[["Deck", "Cabin_num", "Side"]] = df["Cabin"].str.split("/", expand=True)
    df = onehot(df,"HomePlanet")
    df = onehot(df,"Destination")
    df = onehot(df,"Deck")
    df = onehot(df,"Side")
    df = df.drop(["Cabin","Cabin_num"], axis=1)
    return df



In [7]:
# random add nan in train_df
keep = noise_df["Transported"]
keepB = noise_df["PassengerId"]
noise_df = noise_df.drop(["Transported","PassengerId"], axis=1)
for key in noise_df.columns:
    for i in range(7572):
        if np.random.randint(0,100) < 10:
            noise_df[key][i] = pd.NA
print(noise_df.isnull().sum().sort_values(ascending=False))
print(noise_df.shape)
print(noise_df.columns)
print(noise_df.info())
# print(train_df.isnull())
print(noise_df.isnull().sum().sort_values(ascending=False))
noise_df["Transported"] = keep
noise_df["PassengerId"] = keepB
noise_df = processData(noise_df,True)
noise_df.to_csv("./dataset/train_noise.csv", index=False)



Destination    714
HomePlanet     689
VIP            652
Cabin          649
CryoSleep      634
dtype: int64
(7736, 5)
Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 7736 entries, 0 to 8692
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HomePlanet   7047 non-null   object
 1   CryoSleep    7102 non-null   object
 2   Cabin        7087 non-null   object
 3   Destination  7022 non-null   object
 4   VIP          7084 non-null   object
dtypes: object(5)
memory usage: 620.7+ KB
None
Destination    714
HomePlanet     689
VIP            652
Cabin          649
CryoSleep      634
dtype: int64


In [8]:
train_df = processData(train_df, isTrain=True)
train_raw_df = processData(train_raw_df, isTrain=True)
train_null_df = processData(train_null_df, isTrain=False)
train_raw_df.to_csv("./dataset/train_raw_preprocessed.csv", index=False)
train_df.to_csv("./dataset/train_preprocessed_clean.csv", index=False)
train_null_df.to_csv("./dataset/test_preprocessed.csv", index=False)